![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg/320px-Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg.png) 

*Alonso Rios Guerra - 100495821 | Guillermo Sancho González - 100495991*


# *__Aprendizaje automático P1: Predicción del abandono de empleados__*

## *__Introducción__*

En esta práctica tenemos como objetivo desarrollar diferentes métodos de aprendizaje automático para predecir el abandono de los trabajadores de una empresa.

Primero de todo empezaremos leyendo los datos que nos proporciona la empresa.

In [109]:
import numpy as np
import pandas as pd

data_path = 'attrition_availabledata_10.csv.gz'

data = pd.read_csv(data_path, compression='gzip', sep = ',')

## *__EDA Simplificado__*



Un EDA es una análisis exploratorio de datos, para organizar los datos, entender su contenido, entender cual son las variables más relevantes y cómo se relacionan unas con otras, determinar qué hacer con los datos faltantes y con los datos atípicos, y finalmente extraer conclusiones acerca de todo este análisis.

Para hacer un eda debemos responder a distintas preguntas:

-   ¿Cuántas instancias y atributos hay?

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

-   ¿Qué atributos tienen valores faltantes y cuántos?

-   ¿Existen columnas constantes o ID?

-   ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?

A continuación le damos respuesta:

-   ¿Cuántas instancias y atributos hay?

In [93]:
print('La forma de la tabla es:')
print('===============================')
print(data.shape)

La forma de la tabla es:
(2940, 27)


El dataset contiene 2940 instancias, 30 atributos y 1 etiqueta (Attrition).

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardinalidad.

In [ ]:
print('Los tipos de atributos son:')
print('================================')
print(data.dtypes)

Los tipos de atributos son:
hrs                        float64
absences                     int64
JobInvolvement               int64
PerformanceRating            int64
EnvironmentSatisfaction    float64
JobSatisfaction            float64
WorkLifeBalance            float64
Age                          int64
BusinessTravel              object
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
Gender                      object
JobLevel                     int64
JobRole                     object
MaritalStatus               object
MonthlyIncome                int64
NumCompaniesWorked         float64
PercentSalaryHike            int64
StockOptionLevel             int64
TotalWorkingYears          float64
TrainingTimesLastYear        int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
YearsWithCurrManager         int64
Attrition                   object
dtype: object


Existen dos tipos de atributos en nuestro dataset: numéricos y categóricos. Dentro de los numéricos encontramos de tipo entero (absences, Age, JobLevel, ...) y de tipo float (hrs, TotalWorkingYears, JobSatisfaction). En cuanto a lo atributos categóricos encontramos algunos como Department, JobRole, MaritalStatus, ... Para entrenar a nuestro modelo nos interesa codificar las variables categóricas y por ello es importante ver como de viable es según su cardinalidad.

In [103]:
columna_cat = data.select_dtypes(include=['object']).columns # Selecciona las columnas categóricas

print('Cardinalidad de los atributos categóricos:')
print('================================')
for col in columna_cat: # Imprime la cardinalidad de cada atributo categórico
    print(f"{col}: {data[col].nunique()} categorías únicas")

Cardinalidad de los atributos categóricos:
BusinessTravel: 3 categorías únicas
Department: 3 categorías únicas
EducationField: 6 categorías únicas
Gender: 2 categorías únicas
JobRole: 9 categorías únicas
MaritalStatus: 3 categorías únicas
Attrition: 2 categorías únicas


Al observar la ejecución del código anterior vemos que la cardinalidad de nuestros atributos categóricos es baja, en un rango de [2-9], y por ello no implicará ningún problema a la hora realizar una codificación como la One-Hot Encoding.

-   ¿Qué atributos tienen valores faltantes y cuántos?

In [101]:
print('Cuántos valores faltan por atributo:')
print('======================================')
sin_valor = data.isnull().sum()  # Cuenta valores nulos por columna
sin_valor = sin_valor[sin_valor > 0]  # Filtra solo los que tienen valores nulos

print(sin_valor)

Cuántos valores faltan por atributo:
EnvironmentSatisfaction    15
JobSatisfaction            12
WorkLifeBalance            29
NumCompaniesWorked         17
TotalWorkingYears           5
dtype: int64


Tras ejecutar el código anterior, obtenemos que existen 5 atributos con valores faltantes. Estos atributos son: 
EnvironmentSatisfaction con 15 faltantes, 
JobSatisfaction con 12 faltantes,
WorkLifeBalance con 29 faltantes,
NumCompaniesWorked con 17 faltantes y
TotalWorkingYears con 5 faltantes.

- ¿Existen columnas constantes o ID?

In [110]:
# 1. Comprobar columnas constantes
constantes = [col for col in data.columns if data[col].nunique() == 1]
print("Columnas constantes:", constantes)

# 2. Comprobar columnas ID
columnas_id = [col for col in data.columns if data[col].nunique() == len(data)]
print("Columnas ID:", columnas_id)


Columnas constantes: ['EmployeeCount', 'Over18', 'StandardHours']
Columnas ID: ['EmployeeID']


Observamos que existen 3 columnas constantes (EmployeeCount, Over18, StandardHours) y una columna ID (EmployeeID)

- ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?
 

En este caso es fácil ver que es un problema de __clasificación__ porque la etiqueta (Attrition) en los datos train solo pueden tener valores 'Yes' o 'No', por lo que es una clase binaria.

In [112]:
print('Comprobar si la clase está desbalanceada:')
print('======================================')
print(data['Attrition'].value_counts())
print()
print(data['Attrition'].value_counts() / data['Attrition'].count())

Comprobar si la clase está desbalanceada:
Attrition
No     2466
Yes     474
Name: count, dtype: int64

Attrition
No     0.838776
Yes    0.161224
Name: count, dtype: float64


Se puede ver que la clase esta bastante desbalanceada: 83.88% No, 16.12% Yes

## *__¿Cómo se va a realizar la evaluación?__*

## *__Preprocesado de los datos__*

Inicialmente se dividen los datos en x e y, donde x son los inputs, e y es la etiqueta

In [83]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split


X = data.drop(columns=['Attrition'])
Y = data['Attrition']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=1/3, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsRegressor

#Definir los pasos en la Pipeline
knn = KNeighborsRegressor()
scaler = StandardScaler()
imputer = SimpleImputer(strategy='mean')

classif = Pipeline([
    ('imputation', imputer),
    ('standardization', scaler),
    ('knn', knn)
])

classif.fit(X_train, Y_train)
y_hat = classif.predict(X_test)

ValueError: Cannot use mean strategy with non-numeric data:
could not convert string to float: 'Travel_Rarely'